In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import nltk
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
av = pd.read_csv('../datasets/aviation_data.csv')

In [3]:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', None)

In [8]:
def custom_lemmatize(word, tag):
    mapper = {
        'J': wordnet.ADJ,
        'V': wordnet.VERB,
        'N': wordnet.NOUN,
        'R': wordnet.ADV
    }
    pos = mapper.get(tag[0])
    
    return wn.lemmatize(word, pos) if pos else word

In [ ]:
# TODO filter down to just non null probable cause with more than one word then retry lemmatize

In [15]:
#av['N'] 
av['ProbableCause'].apply(lambda x : ' '.join([custom_lemmatize(word, tag) for word, tag in nltk.pos_tag(str(x.split()))]))

AttributeError: 'float' object has no attribute 'split'